## Stacks batch 4 - nb 2


This notebook builds a reference database of loci with bowtie and blast, then runs through the second round of the stacks pipeline from pstacks --> populations, for ALL SAMPLES LANES 1 - 4


<br>
### Reference Genome Building
#### 5/2/2017


**Step One:** Build the fasta file for BOWTIE using the loci in the populations genepop file and the sequences in the batch_1.catalog.tags.tsv.gz file

In [1]:
cd ../scripts

/mnt/hgfs/Pacific cod/DataAnalysis/PCod-Korea-repo/scripts


In [2]:
!python genBOWTIEfasta_fromGENEPOP.py \
../stacks_b4/batch_4.genepop \
../stacks_b4/batch_4.catalog.tags.tsv

-----
Reading loci from file:
../stacks_b4/batch_4.genepop
Stacks version 1.44; Genepop version 4.1.3; April 30, 2017

Done reading loci

Using sequences from catalog file:
../stacks_b4/batch_4.catalog.tags.tsv

Writing new fasta file...


In [3]:
!head seqsforBOWTIE.fa

>13
TGCAGGGTGGGGCCCGACGGGTTCGCCGCCTCCAGCTTCAGCCACTGCTCCCTGGGGAGGGCCAAGCAGTCCTGGACACAGACCACGGGGACACACCCATTTCAATGAACACACACACATTCGCACCGACGACGGTGTCGAC
>21
TGCAGGATGCGGGTGGCTGTGCCGTAGGCCGACAGGGCCCCCTGGATGTCCGTCTGCTCCAGGATCTGAGCCAGCTCGATCCACGCCTCCACGTCGTCCGGGTACTGCTCCGTCACCTTCTTCAGGTGACCCTTTGGTGGAG
>26
TGCAGGTGTTGGGTCTAGTACCCTGGTTCTGCTCAGGGTCAGCTAGGCTTTACGTTGAGGCGGCGGGTGTCTGGACCGGAGATGGCTGTGGACTCTGTGTGTTGTTGCACAGGTCTTTCTATTGGGTGCGTTTCGCATTTGG
>27
TGCAGGTGGCGCTGAATGCGCCGACCAGACGAAAGAGGCCAGTTCCTCGGCTTTATAAACATTCCTGACACTCCCCAAGAATAACATGGCTCATAACTGCAGCCTCGGCGATGGCTCGTGAAAAAAATAAAAACATCACCAC
>44
TGCAGGTTGGAGAGACTGAAATAACCCCCAAAATGAGTAATAATGTAAAAGCAGTCAAGTCCCACAAATTGCTTGGACTATTTATTTCATTCCATTGTTTCGTTGATATGCATTATTGAAAAGTTTCAAGCATATGGATTTA


In [5]:
!mv seqsforBOWTIE.fa ../stacks_b4/batch_4_seqsforBOWTIE.fa


**Step Two:** Run BOWTIE filtering from within the stacks folder
In order to run BOWTIE, the program must be held within the folder you are working.

In [6]:
cd ../stacks_b4

/mnt/hgfs/Pacific cod/DataAnalysis/PCod-Korea-repo/stacks_b4


In [7]:
mkdir refgenome

In [8]:
!cp ../../PCod-US-repo/stacks/refgenome/bowtie-1.1.2 \
refgenome/bowtie-1.1.2

cp: omitting directory '../../PCod-US-repo/stacks/refgenome/bowtie-1.1.2'


not sure why that didn't work. manually copied in bowtie folder

In [9]:
cd refgenome

/mnt/hgfs/Pacific cod/DataAnalysis/PCod-Korea-repo/stacks_b4/refgenome


In [11]:
!bowtie-build ../batch_4_seqsforBOWTIE.fa batch_4

Settings:
  Output files: "batch_4.*.ebwt"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 5 (one in 32)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  ../batch_4_seqsforBOWTIE.fa
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 907699
Using parameters --bmax 680775 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 680775 --dcv 1024
Constructing suffix-array element generat

In [12]:
!bowtie -f -v 3 --sam --sam-nohead \
batch_4 \
../batch_4_seqsforBOWTIE.fa \
batch_4_BOWTIEout.sam

# reads processed: 25569
# reads with at least one reported alignment: 25569 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 25569 alignments to 1 output stream(s)


In [13]:
!python ../../scripts/parseBowtie_DD.py \
batch_4_BOWTIEout.sam \
batch_4_BOWTIE_filtered.fa

Number of Bowtie output lines read: 25569
Number of sequences written to output: 25569



<br>
**Step Three:** BLAST filtering

In [14]:
!makeblastdb -in batch_4_BOWTIE_filtered.fa \
-parse_seqids \
-dbtype nucl \
-out batch_4_BOWTIEfilteredDB



Building a new DB, current time: 05/02/2017 14:35:48
New DB name:   /mnt/hgfs/Pacific cod/DataAnalysis/PCod-Korea-repo/stacks_b4/refgenome/batch_4_BOWTIEfilteredDB
New DB title:  batch_4_BOWTIE_filtered.fa
Sequence type: Nucleotide
Keep Linkouts: T
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 25569 sequences in 2.22704 seconds.


In [17]:
!blastn -query batch_4_BOWTIE_filtered.fa \
-db batch_4_BOWTIEfilteredDB \
-out batch_4_BOWTIE_BLAST_filtered

In [18]:
!python ../../scripts/checkBlastResults_DD.py \
batch_4_BOWTIE_BLAST_filtered \
batch_4_BOWTIE_filtered.fa \
batch_4_BOWTIE_BLAST_filtered.fa \
batch_4_BOWTIE_BLAST_output_bad.fa


Identifying which loci are 'good' and 'bad' based on BLAST alignments...
Writing 'good' and 'bad' loci to their respective files...


In [20]:
!grep -c "^>" batch_4_BOWTIE_BLAST_filtered.fa

25221



<br>
**Step Four:** Create the final SAM file containing the reference database of loci

In [21]:
!bowtie-build batch_4_BOWTIE_BLAST_filtered.fa \
batch_4_ref_genome

Settings:
  Output files: "batch_4_ref_genome.*.ebwt"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 5 (one in 32)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  batch_4_BOWTIE_BLAST_filtered.fa
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 895345
Using parameters --bmax 671509 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 671509 --dcv 1024
Constructing suffix-array


<br>
**Step Five:** Align process_radtags output to new reference "genome"
Note- when aligning, you have to choose a -v value, or the number of mismatches allowed between reads. this should match the -M value chosen in ustacks.

In [22]:
cd ../../scripts/

/mnt/hgfs/Pacific cod/DataAnalysis/PCod-Korea-repo/scripts


In [30]:
!head -n 15 RefGenome_BOWTIEalign_genshell5-2.py

###### Generate Shell Script to Align all FastQ Data Files to BOWTIE ref genome ######

## MF 3/9/2017
## Edited 5/2/2017 for Korean Cod Data, for UNZIPPED FASTQ FILES



## At Command Line: python cstacks_populations_genShell.py ARG1 ARG2 ARG3 ARG4 ARG5
##---- ARG1 = complete sample list file
##---- ARG2 = relative path to bowtie ref database, including file name without filetype suffix
##---- ARG3 = relative path to stacks fastq files, output from process_radtags
##---- ARG4 = batch #
##---- ARG5 = relative path to where you want the .sam files to go




In [31]:
cd ../

/mnt/hgfs/Pacific cod/DataAnalysis/PCod-Korea-repo


In [32]:
!mkdir batch_4_wgenome

In [33]:
cd scripts

/mnt/hgfs/Pacific cod/DataAnalysis/PCod-Korea-repo/scripts


In [40]:
!python RefGenome_BOWTIEalign_genshell5-2.py \
PopMap_L1-4.txt \
../stacks_b4/refgenome/batch_4 \
../samplesT142 \
4 \
../batch_4_wgenome

In [41]:
!head RefGenome_BOWTIEalign_batch4.sh

#!/bin/bash

bowtie -q -v 3 -norc --sam ../stacks_b4/refgenome/batch_4 ../samplesT142/PO010715_02.fq ../batch_4_wgenome/PO010715_02.sam
bowtie -q -v 3 -norc --sam ../stacks_b4/refgenome/batch_4 ../samplesT142/PO010715_02_rep.fq ../batch_4_wgenome/PO010715_02_rep.sam
bowtie -q -v 3 -norc --sam ../stacks_b4/refgenome/batch_4 ../samplesT142/PO010715_04.fq ../batch_4_wgenome/PO010715_04.sam
bowtie -q -v 3 -norc --sam ../stacks_b4/refgenome/batch_4 ../samplesT142/PO010715_06.1.fq ../batch_4_wgenome/PO010715_06.1.sam
bowtie -q -v 3 -norc --sam ../stacks_b4/refgenome/batch_4 ../samplesT142/PO010715_08.1.fq ../batch_4_wgenome/PO010715_08.1.sam
bowtie -q -v 3 -norc --sam ../stacks_b4/refgenome/batch_4 ../samplesT142/PO010715_10.1.fq ../batch_4_wgenome/PO010715_10.1.sam
bowtie -q -v 3 -norc --sam ../stacks_b4/refgenome/batch_4 ../samplesT142/PO010715_11.1.fq ../batch_4_wgenome/PO010715_11.1.sam
bowtie -q -v 3 -norc --sam ../stacks_b4/refgenome/batch_4 ../samplesT142/PO010715_12.fq ../batch_4_wge

In [42]:
!chmod +x RefGenome_BOWTIEalign_batch4.sh

In [ ]:
#ran this at the command line
./RefGenome_BOWTIEalign_batch4.sh

Now time for the second running of stacks!



<br>
### pstacks --> populations

This is the second run-through of stacks, from pstacks to populations. This iteration of the pipeline will have a **stack depth of 10** since I am now calling genotypes. 

In [1]:
cd ../scripts

/mnt/hgfs/Pacific cod/DataAnalysis/PCod-Korea-repo/scripts


In [6]:
!head -n 15 pstacks_populations_genShell_5-3.py

###### Generate Shell Script to Run pstacks --> populations ######

## MF 3/10/2017
## For US Cod Data

## Edited by MF 5/3/2017 for Korea PCod Data



## At Command Line: python cstacks_populations_genShell_3-8 ARG1 ARG2
##---- ARG1 = complete sample list file
##---- ARG2 = sample list for building cstacks catalog


#############################################################################


I had to change the paths within the script.

In [9]:
!python pstacks_populations_genShell_5-3.py \
PopMap_L1-4.txt \
samples_for_cstacks.txt

In [12]:
!head pstacks_populations_5-3.sh

#!/bin/bash

#pstacks
pstacks -t sam -f ../batch_4_wgenome/PO010715_02.sam -o ../batch_4_wgenome -i 001 -m 10 -p 6 --model_type bounded 2>> ../batch_4_wgenome/pstacks_out_b4_wgenome
pstacks -t sam -f ../batch_4_wgenome/PO010715_02_rep.sam -o ../batch_4_wgenome -i 002 -m 10 -p 6 --model_type bounded 2>> ../batch_4_wgenome/pstacks_out_b4_wgenome
pstacks -t sam -f ../batch_4_wgenome/PO010715_04.sam -o ../batch_4_wgenome -i 003 -m 10 -p 6 --model_type bounded 2>> ../batch_4_wgenome/pstacks_out_b4_wgenome
pstacks -t sam -f ../batch_4_wgenome/PO010715_06.1.sam -o ../batch_4_wgenome -i 004 -m 10 -p 6 --model_type bounded 2>> ../batch_4_wgenome/pstacks_out_b4_wgenome
pstacks -t sam -f ../batch_4_wgenome/PO010715_08.1.sam -o ../batch_4_wgenome -i 005 -m 10 -p 6 --model_type bounded 2>> ../batch_4_wgenome/pstacks_out_b4_wgenome
pstacks -t sam -f ../batch_4_wgenome/PO010715_10.1.sam -o ../batch_4_wgenome -i 006 -m 10 -p 6 --model_type bounded 2>> ../batch_4_wgenome/pstacks_out_b4_wgenome
pstacks 

In [13]:
!chmod +x pstacks_populations_5-3.sh

In [ ]:
#run at command line
./pstacks_populations_5-3.sh